<a href="https://colab.research.google.com/github/sheetalkbc/MyColab-Notebooks/blob/master/SentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

pd.set_option("display.max_colwidth", 900)

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/sheetalkbc/TestDataFiles/main/TruckRoll_Alarms.csv?token=AJU6QFHZL7MTBTMFQTFMQ43A62OPK')
#df.drop('Unnamed: 0',inplace=True,axis=1)

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

In [6]:
df['Text1'] = df.WORK_LOG + df.DESCRIPTION

In [7]:
df.labels.value_counts()

N    4393
Y      64
Name: labels, dtype: int64

In [8]:
batch_size = 64
features_1 = np.empty((0,768), float)

for i in tqdm(range(0, df.shape[0], batch_size)):
    batch = df.Text1.tolist()[i:i+batch_size]
    feature_list = model.encode(batch)
    Arr = np.array(feature_list)
    features_1 = np.append(features_1,Arr,axis=0)

print('First Text', np.shape(features_1))

##################

batch_size = 64
features_2 = np.empty((0,768), float)

for i in tqdm(range(0, df.shape[0], batch_size)):
    batch = df.Alarms.tolist()[i:i+batch_size]
    feature_list = model.encode(batch)
    Arr = np.array(feature_list)
    features_2 = np.append(features_2,Arr,axis=0)

print('First Text', np.shape(features_2))

  0%|          | 0/70 [00:00<?, ?it/s]

First Text (4457, 768)


100%|██████████| 70/70 [00:27<00:00,  2.52it/s]

First Text (4457, 768)


In [9]:
print('First Text', np.shape(features_1))
print('Second Text', np.shape(features_2))

First Text (4457, 768)
Second Text (4457, 768)


In [10]:
Combi_Feature = np.stack([features_1,features_2],axis=1)
print(np.shape(Combi_Feature))

Combi_Feature = np.reshape(Combi_Feature,(4457,1536))
print(np.shape(Combi_Feature))

(4457, 2, 768)
(4457, 1536)


In [23]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
  
# Load data in X
db = DBSCAN(eps=5, min_samples=10).fit(Combi_Feature)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
Cluster_labels = db.labels_
  
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(Cluster_labels)) - (1 if -1 in Cluster_labels else 0)
  
print()

print('Clusters \n',np.unique(Cluster_labels,return_counts=True),'\n')
  
df['cluster'] = pd.Series(Cluster_labels)
print('Cluster Count', df.cluster.nunique())
df.loc[df.labels=='Y','cluster'].value_counts()


Clusters 
 (array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22]), array([3008,  493,   41,  110,   10,   43,  145,  142,   10,   57,   17,
         13,   19,  120,   30,   28,   33,   15,   20,   31,   29,   15,
         13,   15])) 

Cluster Count 24


-1     52
 0      8
 21     1
 17     1
 8      1
 6      1
Name: cluster, dtype: int64

In [ ]:
#df.loc[df.cluster==20].labels

In [24]:

#df_filter.loc[df.labels==1,'cluster'].value_counts()

df.loc[df.cluster==0,['INCIDENT_NUMBER','labels','Text1','Alarms']].sort_values(by=['labels'],ascending=False).head(20)



,INCIDENT_NUMBER,labels,Text1,Alarms
118,INC000001691619,Y,cell operation degraded unitname antennaline rf_ext rmod_r major alarm notification additionalfaultid lncel transmission path failure melden alarmen original too activ bitte ort pruffenmelden major alarm,cell notification additionalfaultid lncel transmission path failure _ nms n no_corbanokia_netact gpp_v info batteriewuerdig nan operation degraded unitname antennaline smod_r rf_ext rmod_r major alarm unitname_antennaline anms
1525,INC000001702998,Y,cell operation bbmod_r rf rm minor alarm unitname antennaline original ueberprueft alarme aktiv antl runtime object antl_r port label antenna description tx capable true dc voltage hdlc communication allowed current value threshold major restart helfen bitte ort pruefen entstoeren crq findensporadisch lncel minor alarm,cell operation degraded bbmod_r rf rmod_r additionalf antenna line device failure degradedmrbts _ smod_r bbext additionalfnms n no_corbanokia_netact gpp_v info batteriewuerdig nan mrbts lnbts unitname rd party rm rmnms antennaline minor alarm
1036,INC000001698668,Y,mrbts lnbts base station notification unitname tilt path smod_r rf_ext rmod_r ald_r re antenna line device failure melden sporadisch alarme stehen aktiv ot reset helfen bitte ort pruefen entstoeren closed info anlage alarmfrei on air erkennen konfiguration überprüfen fehler feststellen durchführen fehlerfrei ticket schließenmelden sporadisch antenna line device failure,base station notification unitname tilt path smod_r rf_ext rmod_r ald_r re antenna line device failure notificationmrbts lnbts unitname_tilt _ renms n west_corbanokia_netact gpp_v info batteriewuerdig nan mrbts cell lncel operation degraded degradedmrbts
879,INC000001697357,Y,mrbts lnbts base station notification unitname antennaline path smod_r bbext bbmod_r rf rm antenna line failure melden alarme stehen aktiv ot aktion on site bitte ort pruefen relevante crq findenmelden antenna line failure,base station notification unitname smod_r bbext bbmod_r rf rm antenna line failure dedicated notificationmrbts lnbts _ rmnms n no_corbanokia_netact gpp_v info batteriewuerdig nan mrbts
3889,INC000001721496,Y,cell operation degraded unitname antennaline rf_ext rmod_r antenna line failure melden alarme uberprueft aktiv original bitte ort pruefen entstoeren relevante incs crqs findenmelden antenna line failure,base station notification unitname additionalfaultid mrbts configuration change rejected by bts notificationmrbts lnbts unitname_fsmf _ nms n west_corbanokia_netact gpp_v info batteriewuerdig nan antennaline path smod_r rf_ext rmod_r dc voltage administratively disabled unitname_antennaline anms cell operation degraded antenna line failure degradedmrbts lncel
2324,INC000001709693,Y,cell operation bbmod_r rf rm minor alarm unitname antennaline lncel melden degraded alarme stehen aktiv netact bitte ort prüfen entstoeren closed findenlncel melden minor alarm,cell operation degraded unitname tilt path smod_r rf_ext rmod_r ald_r re antenna line device failure degradedmrbts lnbts lncel unitname_tilt _ renms n west_corbanokia_netact gpp_v info batteriewuerdig nan mrbts antennaline bbmod_r rf rm minor alarm unitname_antennaline bbext rmnms
3739,INC000001720239,Y,cell operation path bbmod_r rf rmod_r re antenna line device failure unitname tilt melden lncel degraded alarmen original tool activ bitte ort pruffen relevante crq findenmelden lncel antenna line device failure,cell operation degraded unitname tilt path smod_r bbext bbmod_r rf rmod_r antenna line device failure unitname_tilt _ annms n west_corbanokia_netact gpp_v info batteriewuerdig nan
1695,INC000001704140,Y,cell operation path smod_r rf_ext rmod_r ald_r re antenna line device failure unitname tilt melden lncel degraded alarmen original tool activ reset helfen bitte ort pruffen alarm aktiv sektor erneut auftreten tauschenmelden lncel antenna line device failure,cell operation degraded unitname tilt path smod_r rf_ext rmod_r ald_r re ante

In [ ]:
# Plot result
import matplotlib.pyplot as plt
  
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = ['y', 'b', 'g', 'r']
print(colors)
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = 'k'
  
    class_member_mask = (labels == k)
  
    xy = features[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                                      markeredgecolor='k', 
                                      markersize=6)
  
    xy = features[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
                                      markeredgecolor='k',
                                      markersize=6)
  
plt.title('number of clusters: %d' %n_clusters_)
plt.show()